In [1]:
!pip install pdfplumber tiktoken neo4j openai  sentence-transformers transformers 


In [2]:
import requests
import os

remote_pdf_url = "https://arxiv.org/pdf/1709.00666.pdf"
pdf_filename = "ch02-downloaded.pdf"

response = requests.get(remote_pdf_url)

if response.status_code == 200:
    with open(pdf_filename, "wb") as pdf_file:
        pdf_file.write(response.content)
else:
    print("Failed to download the PDF. Status code:", response.status_code)

In [3]:
import pdfplumber

text = ""

with pdfplumber.open(pdf_filename) as pdf:
    for page in pdf.pages:
        text += page.extract_text()

print(text[0:20])

Einstein’s Patents a


In [17]:
!pip install python-dotenv


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
# from neo4j import GraphDatabase

# uri = "neo4j+s://5db7d1f6.databases.neo4j.io"  # from Aura
# user = "neo4j"                                 # or your Aura user
# password = ""           # reset from Aura console

# driver = GraphDatabase.driver(
#     uri,
#     auth=(user, password),
#     notifications_min_severity="OFF",
# )

# with driver.session() as session:
#     res = session.run("RETURN 1 AS ok").single()
#     print("Neo4j connected:", res["ok"])


In [19]:
from utils import chunk_text

chunks = chunk_text(text, 500, 40)
print(len(chunks))
print(chunks[0])

89
Einstein’s Patents and Inventions
Asis Kumar Chaudhuri
Variable Energy Cyclotron Centre
1‐AF Bidhan Nagar, Kolkata‐700 064
Abstract: Times magazine selected Albert Einstein, the German born Jewish Scientist as the person of the 20th
century. Undoubtedly, 20th century was the age of science and Einstein’s contributions in unravelling mysteries
of nature was unparalleled. However, few are aware that Einstein was also a great inventor. He and his
collaborators had patented a wide variety of inventions


In [20]:
# %load_ext dotenv
# %dotenv

# import os
# from openai import OpenAI


# open_ai_client = OpenAI(
#     api_key=os.environ.get("OPENAI_API_KEY"),
# )

In [21]:
from sentence_transformers import SentenceTransformer

# Load a local embedding model (no API needed)
local_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def embed(texts):
    """
    Work exactly like your OpenAI embed() function,
    but using a free, local embedding model.
    """
    return local_model.encode(texts).tolist()


In [22]:
embeddings = embed(chunks)

print(embeddings[0][0:3])          # preview numbers
print(len(embeddings))             # number of chunks
print(len(embeddings[0]))          # should print 384


[-0.04776293784379959, 0.06253523379564285, -0.04059494659304619]
89
384


In [26]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

load_dotenv()  # Ensure .env is loaded

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD),
    notifications_min_severity="OFF"
)


In [27]:
driver.execute_query("""CREATE VECTOR INDEX pdf IF NOT EXISTS
FOR (c:Chunk)
ON c.embedding""")

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x733eeae1f4a0>, keys=[])

In [28]:
# Add to neo4j
cypher_query = '''
WITH $chunks as chunks, range(0, size($chunks)) AS index
UNWIND index AS i
WITH i, chunks[i] AS chunk, $embeddings[i] AS embedding
MERGE (c:Chunk {index: i})
SET c.text = chunk, c.embedding = embedding
'''

driver.execute_query(cypher_query, chunks=chunks, embeddings=embeddings)

EagerResult(records=[], summary=<neo4j._work.summary.ResultSummary object at 0x733eeb0b4590>, keys=[])

In [29]:
records, _, _ = driver.execute_query("MATCH (c:Chunk) WHERE c.index = 0 RETURN c.embedding, c.text")

print(records[0]["c.text"][0:30])
print(records[0]["c.embedding"][0:3])

Einstein’s Patents and Inventi
[-0.04776293784379959, 0.06253523379564285, -0.04059494659304619]


In [30]:
question = "At what time was Einstein really interested in experimental works?"
question_embedding = embed([question])[0]

query = '''
CALL db.index.vector.queryNodes('pdf', $k, $question_embedding) YIELD node AS hits, score
RETURN hits.text AS text, score, hits.index AS index
'''
similar_records, _, _ = driver.execute_query(query, question_embedding=question_embedding, k=4)

for record in similar_records:
    print(record["text"])
    print(record["score"], record["index"])
    print("======")

Albert Einstein topped the list.
Einstein’s choice as the person of the century didn’t invoke any resentment, it was generally agreed
that 20th century is the age of Science and undoubtedly, Einstein’s contribution to Science, to the
understanding of the intricate laws of nature was unparalleled. He greatly influenced modern science;
altered our views on space‐time, matter and energy, gave new interpretation to gravity etc. The
enormous popularity he enjoyed during his lifetime and even now, is rare for any individual; religious
leader, politician,
0.8008778095245361 2
Einstein’s life was rather featureless. He diligently worked at the patent office,
played violin, discussed physics with his friends, write few not so interesting papers. Then in 1905, he
took the academic world by surprise. In the annals of physics, the year 1905 is known as “annus
mirabilis” or the year of miracle. Indeed, a miracle happened. Albert Einstein, barely 26 years old,
sitting in an obscure Swiss patent offi

In [31]:
# system_message = "You're en Einstein expert, but can only use the provided documents to respond to the questions."
# user_message = f"""
# Use the following documents to answer the question that will follow:
# {[doc["text"] for doc in similar_records]}

# ---

# The question to answer using information only from the above documents: {question}
# """

# print("Question:", question)

# stream = open_ai_client.chat.completions.create(
#     model="gpt-4",
#     messages=[
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": user_message}
#     ],
#     stream=True,
# )
# for chunk in stream:
#     print(chunk.choices[0].delta.content or "", end="")

In [ ]:
!pip install transformers accelerate


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


: 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "microsoft/Phi-3-mini-4k-instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, dtype=torch.float16)

def hf_chat(prompt: str):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=500)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
try :
    driver.execute_query(f"CREATE FULLTEXT INDEX ftPdfChunk FOR (c:Chunk) ON EACH [c.text]")
except:
    print("Fulltext Index already exists")

In [ ]:
hybrid_query = '''
CALL {
    // vector index
    CALL db.index.vector.queryNodes('pdf', $k, $question_embedding) YIELD node, score
    WITH collect({node:node, score:score}) AS nodes, max(score) AS max
    UNWIND nodes AS n
    // We use 0 as min
    RETURN n.node AS node, (n.score / max) AS score
    UNION
    // keyword index
    CALL db.index.fulltext.queryNodes('ftPdfChunk', $question, {limit: $k})
    YIELD node, score
    WITH collect({node:node, score:score}) AS nodes, max(score) AS max
    UNWIND nodes AS n
    // We use 0 as min
    RETURN n.node AS node, (n.score / max) AS score
}
// dedup
WITH node, max(score) AS score ORDER BY score DESC LIMIT $k
RETURN node, score
'''
similar_hybrid_records, _, _ = driver.execute_query(hybrid_query, question_embedding=question_embedding, question=question, k=4)

for record in similar_hybrid_records:
    print(record["node"]["text"])
    print(record["score"], record["node"]["index"])
    print("======")

CH‐Switzerland
Considering Einstein’s upbringing, his interest in inventions and patents was not unusual.
Being a manufacturer’s son, Einstein grew upon in an environment of machines and instruments.
When his father’s company obtained the contract to illuminate Munich city during beer festival, he
was actively engaged in execution of the contract. In his ETH days Einstein was genuinely interested
in experimental works. He wrote to his friend, “most of the time I worked in the physical laboratory,
fascinated by the direct contact with observation.” Einstein's
1.0 42
Einstein
left his job at the Patent office and joined the University of Zurich on October 15, 1909. Thereafter, he
continued to rise in ladder. In 1911, he moved to Prague University as a full professor, a year later, he
was appointed as full professor at ETH, Zurich, his alma‐mater. In 1914, he was appointed Director of
the Kaiser Wilhelm Institute for Physics (1914–1932) and a professor at the Humboldt University of
Berlin

In [ ]:
# user_message = f"""
# Use the following documents to answer the question that will follow:
# {[doc["node"]["text"] for doc in similar_hybrid_records]}

# ---

# The question to answer using information only from the above documents: {question}
# """

# print("Question:", question)

# stream = open_ai_client.chat.completions.create(
#     model="gpt-4",
#     messages=[
#         {"role": "system", "content": system_message},
#         {"role": "user", "content": user_message}
#     ],
#     stream=True,
# )
# for chunk in stream:
#     print(chunk.choices[0].delta.content or "", end="")

Question: At what time was Einstein really interested in experimental works?
Einstein was genuinely interested in experimental works during his days at ETH, as indicated in the provided documents.

In [ ]:
from utils import chat  # or whatever file you put chat() in

user_message = f"""
Use the following documents to answer the question that will follow:
{[doc["node"]["text"] for doc in similar_hybrid_records]}

---

The question to answer using information only from the above documents: {question}
"""

print("Question:", question)

answer = chat(
    messages=[
        {"role": "system", "content": system_message},
        {"role": "user",  "content": user_message},
    ]
)

print(answer)
